# Phase 5 & 6: Risk Identification & Improvement Proposals

This notebook contains the analysis for the Risk & Portfolio Interaction study.

Phase 5 & 6: Identify Main Sources of Risk & Propose Improvements
=================================================================
Goal: Synthesize findings and propose concrete risk-adjusted return improvements

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("=" * 80)
print("PHASE 5: IDENTIFY MAIN SOURCES OF RISK")
print("=" * 80)

# Load all data
daily_pnl = pd.read_csv('daily_pnl_per_cluster.csv', index_col=0, parse_dates=True)
risk_metrics = pd.read_csv('cluster_risk_metrics.csv', index_col=0)
correlation_matrix = pd.read_csv('correlation_matrix.csv', index_col=0)
trades_df = pd.read_csv('trades_with_clusters.csv')

cluster_cols = [col for col in daily_pnl.columns if 'cluster' in col and 'total' not in col]
cluster_pnl = daily_pnl[cluster_cols]

PHASE 5: IDENTIFY MAIN SOURCES OF RISK


In [ ]:
# COMPREHENSIVE RISK SOURCE ANALYSIS

In [11]:
print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║                        MAIN SOURCES OF RISK                                  ║
╚══════════════════════════════════════════════════════════════════════════════╝
""")

print("""
1. DIRECTIONAL RISK (SINGLE-ASSET XAUUSD)
   ─────────────────────────────────────────
   • All 4 clusters trade ONLY XAUUSD (Gold)
   • Portfolio is 100% concentrated in one asset
   • Large moves in gold affect ALL strategies simultaneously
   • No cross-asset diversification available
   
   QUANTIFIED IMPACT:
   """)
# Calculate correlation of each cluster with total portfolio
for col in cluster_cols:
    corr_with_total = daily_pnl[col].corr(daily_pnl['total_pnl'])
    print(f"      {col.replace('_pnl', '')} corr with portfolio: {corr_with_total:.3f}")

print("""
   
2. REGIME RISK (TREND/VOLATILITY CLUSTERING)
   ─────────────────────────────────────────
   • Cluster 0 & 1: Trend-following, high-ADX environments
   • Cluster 2 & 3: Appear to work in specific market conditions
   • Regime changes can cause multiple clusters to underperform together
   
   QUANTIFIED IMPACT (Sharpe by Cluster):""")
for col in cluster_cols:
    sharpe = risk_metrics.loc[col, 'Annualized Sharpe']
    print(f"      {col.replace('_pnl', '')}: {sharpe:.3f}")

print("""
   
3. CORRELATION RISK (LIMITED DIVERSIFICATION)
   ─────────────────────────────────────────
   • While correlations are generally low, they can increase during stress
   • Some cluster pairs show regime-dependent correlation spikes
   
   CORRELATION MATRIX:""")
print(correlation_matrix.round(3).to_string())

print("""
   
4. TAIL RISK (LEFT SKEW / BIG LOSERS)
   ─────────────────────────────────────────
   • Some clusters show negative skewness (left tail)
   • Fat tails (high kurtosis) indicate extreme events
   
   TAIL RISK METRICS:""")
for col in cluster_cols:
    skew = risk_metrics.loc[col, 'Skewness']
    kurt = risk_metrics.loc[col, 'Kurtosis']
    cvar = risk_metrics.loc[col, 'CVaR 95%']
    skew_status = "  LEFT TAIL" if skew < -0.5 else "✓ Right tail" if skew > 0.5 else "~ Symmetric"
    print(f"      {col.replace('_pnl', '')}: Skew={skew:.2f} {skew_status}, Kurt={kurt:.2f}, CVaR={cvar:.2f}")

print("""
   
5. TIME-OF-DAY / EVENT RISK
   ─────────────────────────────────────────
   • Performance varies significantly by session
   • Certain days/times coincide with economic data releases
   • Thin liquidity periods increase slippage risk
   
6. OVERLAP / LEVERAGE RISK
   ─────────────────────────────────────────
   • Up to 4 trades can be open simultaneously
   • 47% of time has multiple overlapping positions
   • During high overlap, Cluster 1 dominates (73.8%)
   • Stacked exposure amplifies drawdowns
""")


╔══════════════════════════════════════════════════════════════════════════════╗
║                        MAIN SOURCES OF RISK                                  ║
╚══════════════════════════════════════════════════════════════════════════════╝


1. DIRECTIONAL RISK (SINGLE-ASSET XAUUSD)
   ─────────────────────────────────────────
   • All 4 clusters trade ONLY XAUUSD (Gold)
   • Portfolio is 100% concentrated in one asset
   • Large moves in gold affect ALL strategies simultaneously
   • No cross-asset diversification available
   
   QUANTIFIED IMPACT:
   
      cluster_0 corr with portfolio: 0.726
      cluster_1 corr with portfolio: 0.634
      cluster_2 corr with portfolio: 0.440
      cluster_3 corr with portfolio: 0.263

   
2. REGIME RISK (TREND/VOLATILITY CLUSTERING)
   ─────────────────────────────────────────
   • Cluster 0 & 1: Trend-following, high-ADX environments
   • Cluster 2 & 3: Appear to work in specific market conditions
   • Regime changes can cause multiple clust

In [ ]:
# PHASE 6: IMPROVEMENT PROPOSALS

In [3]:
print("\n" + "=" * 80)
print("PHASE 6: PROPOSALS FOR BETTER RISK-ADJUSTED RETURN")
print("=" * 80)


PHASE 6: PROPOSALS FOR BETTER RISK-ADJUSTED RETURN


In [ ]:
# 6.1 CAPITAL RE-ALLOCATION BETWEEN CLUSTERS

In [12]:
print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║           IMPROVEMENT 1: RE-ALLOCATE CAPITAL BETWEEN CLUSTERS                ║
╚══════════════════════════════════════════════════════════════════════════════╝
""")

# Current equal weights
n_clusters = 4
equal_weights = np.array([0.25, 0.25, 0.25, 0.25])

# Proposed weights based on Sharpe
sharpe_values = np.array([risk_metrics.loc[col, 'Annualized Sharpe'] for col in cluster_cols])
sharpe_weights = sharpe_values / sharpe_values.sum()

# Risk-based weights (inverse volatility)
vol_values = np.array([risk_metrics.loc[col, 'Std Dev (Volatility)'] for col in cluster_cols])
inv_vol_weights = (1/vol_values) / (1/vol_values).sum()

print("Current (Equal Weight) Allocation:")
for i, col in enumerate(cluster_cols):
    print(f"   {col.replace('_pnl', '')}: {equal_weights[i]*100:.1f}%")

print("\nProposed Sharpe-Based Allocation:")
for i, col in enumerate(cluster_cols):
    print(f"   {col.replace('_pnl', '')}: {sharpe_weights[i]*100:.1f}%")

print("\nProposed Inverse-Volatility Allocation:")
for i, col in enumerate(cluster_cols):
    print(f"   {col.replace('_pnl', '')}: {inv_vol_weights[i]*100:.1f}%")

# Calculate portfolio metrics for different allocations
cov_matrix = cluster_pnl.cov().values
mean_returns = cluster_pnl.mean().values

def calc_portfolio_metrics(weights):
    port_return = np.sum(weights * mean_returns)
    port_vol = np.sqrt(weights.T @ cov_matrix @ weights)
    port_sharpe = (port_return / port_vol) * np.sqrt(252) if port_vol > 0 else 0
    return port_return, port_vol, port_sharpe

# Compare allocations
allocations = {
    'Equal Weight': equal_weights,
    'Sharpe-Based': sharpe_weights,
    'Inverse-Vol': inv_vol_weights,
    'Focus on C0+C1 (70/30)': np.array([0.40, 0.30, 0.15, 0.15])
}

print("\n Portfolio Performance Comparison:")
print("-" * 70)
print(f"{'Allocation':<25} {'Daily Return':>12} {'Daily Vol':>12} {'Ann. Sharpe':>12}")
print("-" * 70)

for name, weights in allocations.items():
    ret, vol, sharpe = calc_portfolio_metrics(weights)
    print(f"{name:<25} ${ret:>11.2f} ${vol:>11.2f} {sharpe:>12.2f}")


╔══════════════════════════════════════════════════════════════════════════════╗
║           IMPROVEMENT 1: RE-ALLOCATE CAPITAL BETWEEN CLUSTERS                ║
╚══════════════════════════════════════════════════════════════════════════════╝

Current (Equal Weight) Allocation:
   cluster_0: 25.0%
   cluster_1: 25.0%
   cluster_2: 25.0%
   cluster_3: 25.0%

Proposed Sharpe-Based Allocation:
   cluster_0: 43.2%
   cluster_1: 40.5%
   cluster_2: 8.3%
   cluster_3: 8.0%

Proposed Inverse-Volatility Allocation:
   cluster_0: 16.8%
   cluster_1: 22.4%
   cluster_2: 25.0%
   cluster_3: 35.8%

 Portfolio Performance Comparison:
----------------------------------------------------------------------
Allocation                Daily Return    Daily Vol  Ann. Sharpe
----------------------------------------------------------------------
Equal Weight              $       6.66 $      15.48         6.83
Sharpe-Based              $      10.20 $      22.75         7.11
Inverse-Vol               $      

In [ ]:
# 6.2 VOLATILITY TARGETING

In [13]:
print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║           IMPROVEMENT 2: VOLATILITY TARGETING PER CLUSTER                    ║
╚══════════════════════════════════════════════════════════════════════════════╝
""")

target_vol = 20  # Target daily volatility per cluster contribution

print(f"Target Daily Volatility per Cluster: ${target_vol}")
print("\nVolatility-Targeted Weights:")

vol_target_weights = []
for i, col in enumerate(cluster_cols):
    cluster_vol = vol_values[i]
    target_weight = target_vol / cluster_vol if cluster_vol > 0 else 0
    vol_target_weights.append(target_weight)
    print(f"   {col.replace('_pnl', '')}: Vol={cluster_vol:.2f}, Scale Factor={target_weight:.2f}")

# Normalize to sum to 1
vol_target_weights = np.array(vol_target_weights)
vol_target_weights = vol_target_weights / vol_target_weights.sum()

print("\nNormalized Volatility-Targeted Weights:")
for i, col in enumerate(cluster_cols):
    print(f"   {col.replace('_pnl', '')}: {vol_target_weights[i]*100:.1f}%")

ret, vol, sharpe = calc_portfolio_metrics(vol_target_weights)
print(f"\nPortfolio Performance with Vol-Targeting: Sharpe = {sharpe:.2f}")


╔══════════════════════════════════════════════════════════════════════════════╗
║           IMPROVEMENT 2: VOLATILITY TARGETING PER CLUSTER                    ║
╚══════════════════════════════════════════════════════════════════════════════╝

Target Daily Volatility per Cluster: $20

Volatility-Targeted Weights:
   cluster_0: Vol=38.21, Scale Factor=0.52
   cluster_1: Vol=28.72, Scale Factor=0.70
   cluster_2: Vol=25.64, Scale Factor=0.78
   cluster_3: Vol=17.93, Scale Factor=1.12

Normalized Volatility-Targeted Weights:
   cluster_0: 16.8%
   cluster_1: 22.4%
   cluster_2: 25.0%
   cluster_3: 35.8%

Portfolio Performance with Vol-Targeting: Sharpe = 6.26


In [ ]:
# 6.3 RISK PARITY OPTIMIZATION

In [14]:
print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║           IMPROVEMENT 3: RISK PARITY ALLOCATION                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
""")

def risk_parity_objective(weights, cov_matrix):
    """Objective function for risk parity: minimize difference in risk contributions"""
    weights = np.array(weights)
    portfolio_vol = np.sqrt(weights.T @ cov_matrix @ weights)
    marginal_contrib = cov_matrix @ weights
    risk_contrib = weights * marginal_contrib / portfolio_vol
    # We want equal risk contribution
    target_risk = portfolio_vol / len(weights)
    return np.sum((risk_contrib - target_risk)**2)

# Optimize for risk parity
x0 = equal_weights
constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
bounds = [(0.05, 0.5) for _ in range(n_clusters)]  # Min 5%, Max 50%

result = minimize(risk_parity_objective, x0, args=(cov_matrix,), 
                  method='SLSQP', bounds=bounds, constraints=constraints)

risk_parity_weights = result.x

print("Risk Parity Weights:")
for i, col in enumerate(cluster_cols):
    print(f"   {col.replace('_pnl', '')}: {risk_parity_weights[i]*100:.1f}%")

# Calculate risk contribution
portfolio_vol_rp = np.sqrt(risk_parity_weights.T @ cov_matrix @ risk_parity_weights)
mcr = (cov_matrix @ risk_parity_weights) / portfolio_vol_rp
tcr = risk_parity_weights * mcr

print("\nRisk Contribution with Risk Parity:")
for i, col in enumerate(cluster_cols):
    pct = (tcr[i] / portfolio_vol_rp) * 100
    print(f"   {col.replace('_pnl', '')}: {pct:.1f}% of portfolio risk")

ret, vol, sharpe = calc_portfolio_metrics(risk_parity_weights)
print(f"\nPortfolio Performance with Risk Parity: Sharpe = {sharpe:.2f}")


╔══════════════════════════════════════════════════════════════════════════════╗
║           IMPROVEMENT 3: RISK PARITY ALLOCATION                              ║
╚══════════════════════════════════════════════════════════════════════════════╝

Risk Parity Weights:
   cluster_0: 16.3%
   cluster_1: 20.5%
   cluster_2: 25.4%
   cluster_3: 37.8%

Risk Contribution with Risk Parity:
   cluster_0: 25.0% of portfolio risk
   cluster_1: 25.0% of portfolio risk
   cluster_2: 25.0% of portfolio risk
   cluster_3: 25.0% of portfolio risk

Portfolio Performance with Risk Parity: Sharpe = 6.10


In [ ]:
# 6.4 FILTER-BASED IMPROVEMENTS

In [15]:
print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║           IMPROVEMENT 4: APPLY TRADING FILTERS                               ║
╚══════════════════════════════════════════════════════════════════════════════╝
""")

print("""
Recommended Filters Based on Analysis:

1. GLOBAL FILTERS (Apply to all clusters):
   • ATR Filter: Avoid trading when ATR > 4.0 (extreme volatility)
   • Session Filter: Reduce exposure during Asian session (00:00-08:00 UTC)
   • Day Filter: Consider reducing exposure on Thursdays (economic data)

2. PER-CLUSTER FILTERS:
   
   Cluster 0 (Range Trading - Best Performer):
   • Trade during: All sessions except Asian
   • ATR condition: ATR < 3.5 (moderate volatility)
   • Best hours: 10:00-16:00 UTC
   
   Cluster 1 (Trend Following - Strong Performer):
   • Trade during: London and NY Overlap
   • ADX condition: ADX > 25 (confirmed trend)
   • Duration: Allow longer holds (momentum trades)
   
   Cluster 2 (Breakout - Lower Performance):
   • Reduce allocation to 10-15%
   • Only trade with clear directional signals
   • Tighter stops due to left-skewed returns
   
   Cluster 3 (Momentum - Lower Performance):
   • Reduce allocation to 10-15%
   • Avoid Thursday trades
   • Session restriction: Avoid Late US session
""")


╔══════════════════════════════════════════════════════════════════════════════╗
║           IMPROVEMENT 4: APPLY TRADING FILTERS                               ║
╚══════════════════════════════════════════════════════════════════════════════╝


Recommended Filters Based on Analysis:

1. GLOBAL FILTERS (Apply to all clusters):
   • ATR Filter: Avoid trading when ATR > 4.0 (extreme volatility)
   • Session Filter: Reduce exposure during Asian session (00:00-08:00 UTC)
   • Day Filter: Consider reducing exposure on Thursdays (economic data)

2. PER-CLUSTER FILTERS:
   
   Cluster 0 (Range Trading - Best Performer):
   • Trade during: All sessions except Asian
   • ATR condition: ATR < 3.5 (moderate volatility)
   • Best hours: 10:00-16:00 UTC
   
   Cluster 1 (Trend Following - Strong Performer):
   • Trade during: London and NY Overlap
   • ADX condition: ADX > 25 (confirmed trend)
   • Duration: Allow longer holds (momentum trades)
   
   Cluster 2 (Breakout - Lower Performance):
   • 

In [ ]:
# 6.5 OVERLAP CONTROLS

In [16]:
print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║           IMPROVEMENT 5: CONTROL TRADE OVERLAP & MAX EXPOSURE                ║
╚══════════════════════════════════════════════════════════════════════════════╝
""")

print("""
Recommended Overlap Controls:

1. MAX SIMULTANEOUS TRADES:
   • Limit to 3 trades max at any time
   • Current max is 4, often running 2+ trades

2. MAX SAME-DIRECTION EXPOSURE:
   • No more than 2 trades in same direction (all long or all short)
   
3. CLUSTER DIVERSITY REQUIREMENT:
   • When >2 trades open, require different clusters
   • Prevents Cluster 1 domination (currently 73.8% during overlap)

4. REGIME-BASED SCALING:
   • In high-volatility regimes: Reduce position size by 25-50%
   • In low-ADX (ranging) markets: Favor Cluster 0, reduce Cluster 1

5. TIME-BASED LIMITS:
   • During major economic releases: No new entries
   • During low-liquidity Asian hours: Reduce max positions to 2
""")


╔══════════════════════════════════════════════════════════════════════════════╗
║           IMPROVEMENT 5: CONTROL TRADE OVERLAP & MAX EXPOSURE                ║
╚══════════════════════════════════════════════════════════════════════════════╝


Recommended Overlap Controls:

1. MAX SIMULTANEOUS TRADES:
   • Limit to 3 trades max at any time
   • Current max is 4, often running 2+ trades

2. MAX SAME-DIRECTION EXPOSURE:
   • No more than 2 trades in same direction (all long or all short)
   
3. CLUSTER DIVERSITY REQUIREMENT:
   • When >2 trades open, require different clusters
   • Prevents Cluster 1 domination (currently 73.8% during overlap)

4. REGIME-BASED SCALING:
   • In high-volatility regimes: Reduce position size by 25-50%
   • In low-ADX (ranging) markets: Favor Cluster 0, reduce Cluster 1

5. TIME-BASED LIMITS:
   • During major economic releases: No new entries
   • During low-liquidity Asian hours: Reduce max positions to 2



In [ ]:
# SUMMARY TABLE

In [17]:
print("\n" + "=" * 80)
print("PORTFOLIO OPTIMIZATION SUMMARY")
print("=" * 80)

summary_data = {
    'Strategy': ['Equal Weight', 'Sharpe-Based', 'Inverse-Vol', 'Risk Parity', 'Focus C0+C1'],
    'C0 Weight': [25.0, sharpe_weights[0]*100, inv_vol_weights[0]*100, risk_parity_weights[0]*100, 40.0],
    'C1 Weight': [25.0, sharpe_weights[1]*100, inv_vol_weights[1]*100, risk_parity_weights[1]*100, 30.0],
    'C2 Weight': [25.0, sharpe_weights[2]*100, inv_vol_weights[2]*100, risk_parity_weights[2]*100, 15.0],
    'C3 Weight': [25.0, sharpe_weights[3]*100, inv_vol_weights[3]*100, risk_parity_weights[3]*100, 15.0],
}

all_weights = [equal_weights, sharpe_weights, inv_vol_weights, risk_parity_weights, 
               np.array([0.40, 0.30, 0.15, 0.15])]
summary_data['Ann. Sharpe'] = [calc_portfolio_metrics(w)[2] for w in all_weights]

summary_df = pd.DataFrame(summary_data)
print(summary_df.round(2).to_string(index=False))

# Save summary
summary_df.to_csv('optimization_summary.csv', index=False)


PORTFOLIO OPTIMIZATION SUMMARY
    Strategy  C0 Weight  C1 Weight  C2 Weight  C3 Weight  Ann. Sharpe
Equal Weight      25.00      25.00      25.00      25.00         6.83
Sharpe-Based      43.23      40.46       8.29       8.02         7.11
 Inverse-Vol      16.80      22.35      25.04      35.81         6.26
 Risk Parity      16.29      20.53      25.38      37.80         6.10
 Focus C0+C1      40.00      30.00      15.00      15.00         7.10


In [ ]:
# VISUALIZATIONS

In [18]:
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Plot 1: Weight Comparison
ax1 = axes[0, 0]
x = np.arange(n_clusters)
width = 0.15
strategies = ['Equal', 'Sharpe', 'InvVol', 'RiskParity', 'Focus']
colors = plt.cm.Set2(np.linspace(0, 1, 5))

for i, (strat, weights) in enumerate(zip(strategies, all_weights)):
    ax1.bar(x + i*width - 2*width, weights*100, width, label=strat, color=colors[i])

ax1.set_xticks(x)
ax1.set_xticklabels([c.replace('_pnl', '') for c in cluster_cols])
ax1.set_ylabel('Weight (%)')
ax1.set_title('Portfolio Weight Comparison', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Sharpe Ratio Comparison
ax2 = axes[0, 1]
sharpe_values_all = [calc_portfolio_metrics(w)[2] for w in all_weights]
colors_sharpe = ['green' if s > sharpe_values_all[0] else 'gray' for s in sharpe_values_all]
bars = ax2.bar(strategies, sharpe_values_all, color=colors_sharpe, alpha=0.7, edgecolor='black')
ax2.axhline(y=sharpe_values_all[0], color='red', linestyle='--', label=f'Baseline: {sharpe_values_all[0]:.2f}')
ax2.set_ylabel('Annualized Sharpe Ratio')
ax2.set_title('Portfolio Sharpe by Strategy', fontsize=14, fontweight='bold')
ax2.legend()
for bar, val in zip(bars, sharpe_values_all):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, f'{val:.2f}', 
             ha='center', fontweight='bold')
ax2.grid(True, alpha=0.3)

# Plot 3: Risk Contribution (Equal vs Risk Parity)
ax3 = axes[1, 0]
# Equal weight risk contribution
portfolio_vol_eq = np.sqrt(equal_weights.T @ cov_matrix @ equal_weights)
mcr_eq = (cov_matrix @ equal_weights) / portfolio_vol_eq
tcr_eq = equal_weights * mcr_eq
tcr_eq_pct = (tcr_eq / portfolio_vol_eq) * 100

# Risk parity risk contribution
tcr_rp_pct = (tcr / portfolio_vol_rp) * 100

x = np.arange(n_clusters)
width = 0.35
ax3.bar(x - width/2, tcr_eq_pct, width, label='Equal Weight', color='steelblue', alpha=0.7)
ax3.bar(x + width/2, tcr_rp_pct, width, label='Risk Parity', color='coral', alpha=0.7)
ax3.axhline(y=25, color='black', linestyle='--', alpha=0.5, label='Target (25%)')
ax3.set_xticks(x)
ax3.set_xticklabels([c.replace('_pnl', '') for c in cluster_cols])
ax3.set_ylabel('Risk Contribution (%)')
ax3.set_title('Risk Contribution: Equal vs Risk Parity', fontsize=14, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Efficient Frontier Approximation
ax4 = axes[1, 1]
# Generate random portfolios
np.random.seed(42)
n_portfolios = 1000
results = []
for _ in range(n_portfolios):
    w = np.random.random(n_clusters)
    w = w / w.sum()
    ret, vol, sharpe = calc_portfolio_metrics(w)
    results.append((ret, vol, sharpe, w))

rets = [r[0] for r in results]
vols = [r[1] for r in results]
sharpes = [r[2] for r in results]

scatter = ax4.scatter(vols, rets, c=sharpes, cmap='viridis', alpha=0.5, s=10)
plt.colorbar(scatter, ax=ax4, label='Sharpe')

# Mark specific portfolios
for name, weights, color, marker in [
    ('Equal', equal_weights, 'red', 's'),
    ('Sharpe', sharpe_weights, 'blue', '^'),
    ('RiskParity', risk_parity_weights, 'green', 'o'),
]:
    ret, vol, sharpe = calc_portfolio_metrics(weights)
    ax4.scatter(vol, ret, color=color, marker=marker, s=150, edgecolor='black', 
                label=f'{name} (SR={sharpe:.2f})', zorder=5)

ax4.set_xlabel('Daily Volatility ($)')
ax4.set_ylabel('Daily Return ($)')
ax4.set_title('Efficient Frontier & Portfolio Strategies', fontsize=14, fontweight='bold')
ax4.legend(loc='lower right')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('phase5_6_optimization.png', dpi=150, bbox_inches='tight')
plt.close()
print(f"\n Phase 5 & 6 visualization saved to: phase5_6_optimization.png")


 Phase 5 & 6 visualization saved to: phase5_6_optimization.png


In [ ]:
# FINAL RECOMMENDATIONS

In [19]:
print("\n" + "=" * 80)
print("FINAL RECOMMENDATIONS FOR BETTER RISK-ADJUSTED RETURN")
print("=" * 80)

print("""
1. IMMEDIATE ACTIONS:
   ✓ Reduce allocation to Clusters 2 & 3 (lower Sharpe, higher tail risk)
   ✓ Increase allocation to Clusters 0 & 1 (higher Sharpe, better behavior)
   ✓ Implement max overlap limit of 3 trades

2. SHORT-TERM IMPROVEMENTS:
   ✓ Apply volatility targeting to normalize risk contribution
   ✓ Implement session-based filters (reduce Asian session exposure)
   ✓ Add ATR-based position sizing (reduce size in high volatility)

3. MEDIUM-TERM ENHANCEMENTS:
   ✓ Move towards risk parity allocation for balanced risk
   ✓ Develop regime-aware allocation rules
   ✓ Implement correlation monitoring for dynamic adjustment

4. EXPECTED IMPROVEMENTS:
   • Current Equal-Weight Sharpe: """ + f"{sharpe_values_all[0]:.2f}" + """
   • Projected Optimized Sharpe: """ + f"{max(sharpe_values_all):.2f}" + """
   • Estimated improvement: """ + f"{(max(sharpe_values_all)/sharpe_values_all[0] - 1)*100:.1f}%" + """ better risk-adjusted return
   • Reduced maximum drawdown through overlap controls
   • More stable returns through volatility targeting
""")

print("\n PHASE 5 & 6 COMPLETED")


FINAL RECOMMENDATIONS FOR BETTER RISK-ADJUSTED RETURN

1. IMMEDIATE ACTIONS:
   ✓ Reduce allocation to Clusters 2 & 3 (lower Sharpe, higher tail risk)
   ✓ Increase allocation to Clusters 0 & 1 (higher Sharpe, better behavior)
   ✓ Implement max overlap limit of 3 trades

2. SHORT-TERM IMPROVEMENTS:
   ✓ Apply volatility targeting to normalize risk contribution
   ✓ Implement session-based filters (reduce Asian session exposure)
   ✓ Add ATR-based position sizing (reduce size in high volatility)

3. MEDIUM-TERM ENHANCEMENTS:
   ✓ Move towards risk parity allocation for balanced risk
   ✓ Develop regime-aware allocation rules
   ✓ Implement correlation monitoring for dynamic adjustment

4. EXPECTED IMPROVEMENTS:
   • Current Equal-Weight Sharpe: 6.83
   • Projected Optimized Sharpe: 7.11
   • Estimated improvement: 4.2% better risk-adjusted return
   • Reduced maximum drawdown through overlap controls
   • More stable returns through volatility targeting


 PHASE 5 & 6 COMPLETED
